In [10]:
import os
from pathlib import Path
import sys
import shutil
import json

In [34]:
def move_eval_json_up(dir_path):
    eval_results_dir = os.listdir(dir_path/"results")[0]
    eval_results_path = dir_path/"results"/eval_results_dir/"eval_results.json"
    shutil.move(eval_results_path, dir_path/"eval_results.json")

In [36]:
def format_model_folder(PATH):

    # list checkpoints
    checkpoints = os.listdir(PATH)
    if "eval_results.json" in checkpoints: checkpoints.remove("eval_results.json")
    if "results" in checkpoints: checkpoints.remove("results")
    checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
    latest_dir = checkpoints[-1]

    # get trainer state
    trainer_state_path = PATH/latest_dir/"trainer_state.json"
    with open(trainer_state_path, "r") as file:
        trainer_state = json.load(file)

    # move trainer state up
    shutil.move(trainer_state_path, PATH/"trainer_state.json")

    best_dir = Path(trainer_state['best_model_checkpoint']).parts[-1]

    # Remove non-best models
    checkpoints.remove(best_dir)
    for dir in checkpoints:
        shutil.rmtree(PATH/dir)

    # move contents of best model up
    for file in os.listdir(PATH/best_dir):
        if file == "trainer_state.json":
            os.remove(PATH/best_dir/file)
        else:
            shutil.move(PATH/best_dir/file, PATH/file)

    # remove empty folder
    shutil.rmtree(PATH/best_dir)

In [32]:
#! ONLY RUN THIS FOR FRESH DNABERT2 RUNS
# ROOT_DIR = Path("results/DNABERT2/alvis/GUE")

# for dir in os.listdir(ROOT_DIR):
#     # Move eval_results.json up
#     move_eval_json_up(dir_path)
    
#     # Delete empty "results" folder
#     shutil.rmtree(dir_path/"results")

#     # save trainer_state.json from latest checkpoint, but model from best checkpoint
#     format_model_folder(ROOT_DIR/dir)